In [2]:
import pyspark
sc = pyspark.SparkContext()
from pyspark.sql import SQLContext
sql = SQLContext(sc)

In [49]:
###Random forest regression

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Load and parse the data file, converting it to a DataFrame.
#data = sql.read.format("libsvm").load("data.txt")
trainingData = sql.read.format("libsvm").load("data.txt")
testData = sql.read.format("libsvm").load("data_2.txt")

#print type(data)
#print data.head(2)


# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=16).fit(data)
    
# Split the data into training and test sets (30% held out for testing)
#(trainingData, testData) = data.randomSplit([1.0, 1.0])


# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures", numTrees=50, maxDepth=5, seed=42)

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(236)


print type(predictions)


#print predictions.select("prediction", "label", "features").head(3)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

rfModel = model.stages[1]
print(rfModel)  # summary only

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
|0.5827045432967333|  0.0|       (2,[1],[1.0])|
|0.5827045432967333|  0.0| (2,[0,1],[1.0,1.0])|
|0.5563156544078445|  0.0| (2,[0,1],[2.0,1.0])|
|0.5704823210745111|  0.0| (2,[0,1],[3.0,1.0])|
|0.5704823210745111|  0.0| (2,[0,1],[4.0,1.0])|
|0.5495378766300667|  1.0| (2,[0,1],[5.0,1.0])|
| 0.551760098852289|  0.0| (2,[0,1],[6.0,1.0])|
| 0.551760098852289|  1.0| (2,[0,1],[7.0,1.0])|
|0.5595378766300667|  0.0| (2,[0,1],[8.0,1.0])|
|0.5495378766300667|  0.0| (2,[0,1],[9.0,1.0])|
|0.5495378766300667|  1.0|(2,[0,1],[10.0,1.0])|
|1.5159252434489405|  1.0|(2,[0,1],[11.0,1.0])|
|2.6029733740242103|  1.0|(2,[0,1],[12.0,1.0])|
|2.6029733740242103|  1.0|(2,[0,1],[13.0,1.0])|
| 4.925717911951567|  7.0|(2,[0,1],[14.0,1.0])|
| 5.938289627187369|  8.0|(2,[0,1],[15.0,1.0])|
| 5.938289627187369|  8.0|(2,[0,1],[16.0,1.0])|
| 6.248481921649014| 11.0|(2,[0,1],[17.0

AttributeError: 'DataFrame' object has no attribute 'mean'